Code pour se connecter à Copernicus pour récupérer des images Sentinel-2 et faire des séries temporelles de ndvi.
Il faut:

-un compte Copernicus

-un geojson de la zone d'intérêt 

jupyter notebook --notebook-dir=D:/ -> permet d'ouvrir le doc mais pas dans C pour avoir plus de stockage

lien vers de la doc pour plus d'informations:
https://sentinelsat.readthedocs.io/en/v0.12.1/api.html

In [1]:
pip install geopandas

In [2]:
pip install sentinelsat

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install rasterio

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install fiona

In [11]:
#importer les librairies et fonctions utiles

import fiona
import geopandas as gpd
from sentinelsat import SentinelAPI,read_geojson,geojson_to_wkt,make_path_filter
from zipfile import ZipFile
import rasterio as rio
import rasterio.mask
from rasterio.plot import show
import numpy as np
import os
import glob
from datetime import date
import matplotlib as pyplot
import requests

AVANT DE LANCER CE BOUT DE CODE:
-penser à importer le geojson de la zone d'intérêt grâce à l'icône qui ressemble à une feuille avec une flèche sur l'interface de gauche (si il n'est pas déjà dans le dossier)

In [12]:
#connexion à Copernicus
api = SentinelAPI('id', 'mdp', 'https://scihub.copernicus.eu/dhus') #penser à changer l'id et le mdp SentinelAPI('id', 'mdp.', 'https://scihub.copernicus.eu/dhus')
#zone d'intérêt
footprint = geojson_to_wkt(read_geojson('decoupe1.geojson')) #penser à changer le nom
#requête de produits (zone d'intérêt, date, plateforme, couverture nuageuse de 0 à 20%)
products = api.query(footprint,
                     date=('20230601', '20230703'), #penser à changer les dates (format aaaammjj) en fonction de vos attentes et des archives
                     platformname='Sentinel-2',
                     processinglevel= 'Level-2A',
                     cloudcoverpercentage=(0, 100))

In [13]:
#créer un data frame pour afficher les produits trouvés
products_df = api.to_dataframe(products)
#on range les produits en fonction de la date
products_df_sorted = products_df.sort_values(['ingestiondate'], ascending=[True])
#on affiche le data frame des produits
products_df_sorted


,title,link,link_alternative,link_icon,summary,ondemand,generationdate,beginposition,endposition,ingestiondate,...,s2datatakeid,producttype,platformidentifier,orbitdirection,platformserialidentifier,processinglevel,datastripidentifier,granuleidentifier,identifier,uuid
fe4728e2-e203-47d2-94d2-27244077d244,S2B_MSIL2A_20230602T105629_N0509_R094_T30TXS_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2023-06-02T10:56:29.024Z, Instrument: MS...",false,2023-06-02 12:30:43,2023-06-02 10:56:29.024,2023-06-02 10:56:29.024,2023-06-02 15:57:52.052,...,GS2B_20230602T105629_032581_N05.09,S2MSI2A,2017-013A,DESCENDING,Sentinel-2B,Level-2A,S2B_OPER_MSI_L2A_DS_2BPS_20230602T123043_S2023...,S2B_OPER_MSI_L2A_TL_2BPS_20230602T123043_A0325...,S2B_MSIL2A_20230602T105629_N0509_R094_T30TXS_2...,fe4728e2-e203-47d2-94d2-27244077d244
5d3b62e0-c722-4ce6-921e-13c0b46ed362,S2B_MSIL2A_20230605T110619_N0509_R137_T30TXS_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2023-06-05T11:06:19.024Z, Instrument: MS...",false,2023-06-05 14:12:26,2023-06-05 11:06:19.024,2023-06-05 11:06:19.024,2023-06-05 16:49:02.890,...,GS2B_20230605T110619_032624_N05.09,S2MSI2A,2017-013A,DESCENDING,Sentinel-2B,Level-2A,S2B_OPER_MSI_L2A_DS_2BPS_20230605T141226_S2023...,S2B_OPER_MSI_L2A_TL_2BPS_20230605T141226_A0326...,S2B_MSIL2A_20230605T110619_N0509_R137_T30TXS_2...,5d3b62e0-c722-4ce6-921e-13c0b46ed362
770488cf-e22b-44e6-a7b2-9fe2fbab92dd,S2A_MSIL2A_20230607T105621_N0509_R094_T30TXS_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2023-06-07T10:56:21.024Z, Instrument: MS...",false,2023-06-07 17:13:12,2023-06-07 10:56:21.024,2023-06-07 10:56:21.024,2023-06-07 18:48:46.733,...,GS2A_20230607T105621_041561_N05.09,S2MSI2A,2015-028A,DESCENDING,Sentinel-2A,Level-2A,S2A_OPER_MSI_L2A_DS_2APS_20230607T171312_S2023...,S2A_OPER_MSI_L2A_TL_2APS_20230607T171312_A0415...,S2A_MSIL2A_20230607T105621_N0509_R094_T30TXS_2...,770488cf-e22b-44e6-a7b2-9fe2fbab92dd
46a80885-8c84-48d0-b5a7-3cb699b1a8a7,S2A_MSIL2A_20230610T110621_N0509_R137_T30TXS_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2023-06-10T11:06:21.024Z, Instrument: MS...",false,2023-06-10 18:53:04,2023-06-10 11:06:21.024,2023-06-10 11:06:21.024,2023-06-10 20:46:17.663,...,GS2A_20230610T110621_041604_N05.09,S2MSI2A,2015-028A,DESCENDING,Sentinel-2A,Level-2A,S2A_OPER_MSI_L2A_DS_2APS_20230610T185304_S2023...,S2A_OPER_MSI_L2A_TL_2APS_20230610T185304_A0416...,S2A_MSIL2A_20230610T110621_N0509_R137_T30TXS_2...,46a80885-8c84-48d0-b5a7-3cb699b1a8a7
7f31fe75-cfb8-42d2-98a3-f5ce0afd2428,S2B_MSIL2A_20230612T105629_N0509_R094_T30TXS_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2023-06-12T10:56:29.024Z, Instrument: MS...",false,2023-06-12 12:29:15,2023-06-12 10:56:29.024,2023-06-12 10:56:29.024,2023-06-12 14:38:42.738,...,GS2B_20230612T105629_032724_N05.09,S2MSI2A,2017-013A,DESCENDING,Sentinel-2B,Level-2A,S2B_OPER_MSI_L2A_DS_2BPS_20230612T122915_S2023...,S2B_OPER_MSI_L2A_TL_2BPS_20230612T122915_A0327...,S2B_MSIL2A_20230612T105629_N0509_R094_T30TXS_2...,7f31fe75-cfb8-42d2-98a3-f5ce0afd2428
147b7a1f-0383-4a53-b439-cd3daf12f4f4,S2B_MSIL2A_20230615T110629_N0509_R137_T30TXS_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2023-06-15T11:06:29.024Z, Instrument: MS...",false,2023-06-15 13:58:07,2023-06-15 11:06:29.024,2023-06-15 11:06:29.024,2023-06-15 17:14:59.461,...,GS2B_20230615T110629_0327

lien vers de la doc pour gérer les fichiers .zip : https://www.geeksforgeeks.org/working-zip-files-python/

In [14]:
nbr_img=len(products_df_sorted)
nbr_img

13

In [3]:
#cette cellule ne fonctionne pas vous n'êtes pas obligé de la lancer, par contre il faudra relancer plusieurs fois le code en changeant la date de départ pour éviter les images qui ne sont pas disponibles
#on va tester les URLs de téléchargement des images si on a l'erreur HTTP status 500 on retire l'image de notre data framede produits

for i in range(0,nbr_img):
    k=products_df_sorted['link'][i]
    print(k)
    try:
        r = requests.head(k)
        print(r.status_code)
    except requests.ConnectionError:
        print(k, " failed to connect")
    if r.status_code == 500:
        products_df_sorted.drop([i], inplace=True)

products_df_sorted

NameError: name 'nbr_img' is not defined

ATTENTION: si vous utilisez ce code vous pouvez avoir l'erreur : "Triggered retrieval from the Long Term Archive". 

Cette erreur signifie qu'une ou plusieurs images ont été archivées, elles ne peuvent donc pas être téléchargées à la volée avec ce code.

on peut aussi avoir cette erreur: "HTTP status 500: NullPointerException : Object (e621a8c6-4f28-4ec1-a663-fcaf1ba2142c.zip)not found"
-> si y a l'erreur HTTP faut relancer le code en enlevant cette image ça veut dire qu'on peut pas la télécharger


J'ai ajouté un "try:" ça permet de pouvoir lancer le code sans avoir à gérer les images archivées mais on a toujours le problème de l'erreur HTTP y a un bout de solution avec le test de l'url mais ça ne prend pas e ncompte la connexion donc on a forcément l'erreur 401.

AVANT DE LANCER CE BOUT DE CODE:

-pensez à créer un sous-dossier et le nommer "resultat_ndvi"

In [1]:
#ouverture du shape pour faire un mask 
with fiona.open('decoupe2.shp','r') as shapefile:
  for feature in shapefile:
    shapes= [feature['geometry']]


NameError: name 'fiona' is not defined

In [16]:
try:
    for i in range(0,nbr_img):
        api.download(products_df_sorted.index[i]) # on télécharge l'image à l'index i

        #récupérer le nom de l'image
        title_img= products_df_sorted.title[i]
        title_img+=".zip"

        #on va déziper le fichier
        with ZipFile(title_img, 'r') as zip:
            zip.printdir()
            print('Extracting all the files now...')
            zip.extractall()
            print('Done!')
        #on met dans une liste toutes les bandes et images que contient le fichier zip
        with ZipFile(title_img) as zip:
            liste= zip.namelist()

        #on récupère la bande 4
        B4 = sorted ([f for f in liste if 'B04' in f]) 
          # on récupère la bande 8
        B8 = sorted ([f for f in liste if 'B08' in f])

        # récuparatin des données de la bande B4
        with rio.open(B4[0]) as src:
            meta = src.meta  # on récupère les métadonnées qui serviront de base pour la création du raster de l'indice
            data_b4 = src.read(1)
            data_b4 = np.where(data_b4 != src.nodata, data_b4, np.nan)  # gestion des valeurs nodata


        # récuparatin des données de la bande B8
        with rio.open(B8[0]) as src:
            meta = src.meta  # on récupère les métadonnées qui serviront de base pour la création du raster de l'indice
            data_b8 = src.read(1)
            data_b8 = np.where(data_b8 != src.nodata, data_b8, np.nan)  # gestion des valeurs nodata

        # Calcul du NDVI
        ndvi = (data_b8 - data_b4) / (data_b8 + data_b4)



        #sauvegarder le résultat dans un fichier
        my_date=str(products_df_sorted.beginposition[i])[:10]
        print(i," , ", my_date)
        my_image="ndvi_esnandes_"+my_date+".tif"
        print(my_image)
        my_dir="resultat_ndvi"
        print(my_dir)
        meta=src.meta
        print(meta)
        meta['dtype']='float32'
        meta['driver']='GTiff'
        with rio.open(my_dir+'/'+ my_image, 'w', **meta) as dst:
            dst.write(ndvi, 1)
        print("meta:", meta) 

        #Mask du ndvi
        with rio.open(my_dir+'/'+ my_image) as src:
            out_image, out_transform= rasterio.mask.mask(src, shapes, crop=True)
            out_meta=src.meta
            out_meta.update({
                  'driver': 'Gtiff',
                  'height': out_image.shape[1],
                  'width' : out_image.shape[2],
                  'transform': out_transform})
        print("out_meta:", out_meta)    


        with rio.open(my_dir+'/'+ my_image, 'w', **out_meta) as dst:
            dst.write(out_image)
except Exception as e: 
    print(e)
    pass


MD5 checksumming:   0%|          | 0.00/1.16G [00:00<?, ?B/s]

File Name                                             Modified             Size
S2B_MSIL2A_20230602T105629_N0509_R094_T30TXS_20230602T123043.SAFE/ 2023-06-02 14:54:00            0
S2B_MSIL2A_20230602T105629_N0509_R094_T30TXS_20230602T123043.SAFE/manifest.safe 2023-06-02 14:54:00        68871
S2B_MSIL2A_20230602T105629_N0509_R094_T30TXS_20230602T123043.SAFE/MTD_MSIL2A.xml 2023-06-02 14:53:50        54888
S2B_MSIL2A_20230602T105629_N0509_R094_T30TXS_20230602T123043.SAFE/INSPIRE.xml 2023-06-02 14:53:50        18609
S2B_MSIL2A_20230602T105629_N0509_R094_T30TXS_20230602T123043.SAFE/rep_info/ 2023-06-02 14:53:50            0
S2B_MSIL2A_20230602T105629_N0509_R094_T30TXS_20230602T123043.SAFE/GRANULE/ 2023-06-02 14:53:50            0
S2B_MSIL2A_20230602T105629_N0509_R094_T30TXS_20230602T123043.SAFE/HTML/ 2023-06-02 14:53:50            0
S2B_MSIL2A_20230602T105629_N0509_R094_T30TXS_20230602T123043.SAFE/DATASTRIP/ 2023-06-02 14:53:50            0
S2B_MSIL2A_20230602T105629_N0509_R094_T30TXS_20230

Done!
0  ,  2023-06-02
ndvi_esnandes_2023-06-02.tif
resultat_ndvi
{'driver': 'JP2OpenJPEG', 'dtype': 'uint16', 'nodata': None, 'width': 10980, 'height': 10980, 'count': 1, 'crs': None, 'transform': Affine(10.0, 0.0, 600000.0,
       0.0, -10.0, 5200020.0)}
meta: {'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 10980, 'height': 10980, 'count': 1, 'crs': None, 'transform': Affine(10.0, 0.0, 600000.0,
       0.0, -10.0, 5200020.0)}
out_meta: {'driver': 'Gtiff', 'dtype': 'float32', 'nodata': None, 'width': 196, 'height': 152, 'count': 1, 'crs': None, 'transform': Affine(10.0, 0.0, 642830.0,
       0.0, -10.0, 5125450.0)}


MD5 checksumming:   0%|          | 0.00/596M [00:00<?, ?B/s]

File Name                                             Modified             Size
S2B_MSIL2A_20230605T110619_N0509_R137_T30TXS_20230605T141226.SAFE/ 2023-06-05 16:19:36            0
S2B_MSIL2A_20230605T110619_N0509_R137_T30TXS_20230605T141226.SAFE/INSPIRE.xml 2023-06-05 16:19:30        18846
S2B_MSIL2A_20230605T110619_N0509_R137_T30TXS_20230605T141226.SAFE/MTD_MSIL2A.xml 2023-06-05 16:19:30        55127
S2B_MSIL2A_20230605T110619_N0509_R137_T30TXS_20230605T141226.SAFE/manifest.safe 2023-06-05 16:19:36        69098
S2B_MSIL2A_20230605T110619_N0509_R137_T30TXS_20230605T141226.SAFE/AUX_DATA/ 2023-06-05 16:19:30            0
S2B_MSIL2A_20230605T110619_N0509_R137_T30TXS_20230605T141226.SAFE/DATASTRIP/ 2023-06-05 16:19:30            0
S2B_MSIL2A_20230605T110619_N0509_R137_T30TXS_20230605T141226.SAFE/GRANULE/ 2023-06-05 16:19:30            0
S2B_MSIL2A_20230605T110619_N0509_R137_T30TXS_20230605T141226.SAFE/rep_info/ 2023-06-05 16:19:30            0
S2B_MSIL2A_20230605T110619_N0509_R137_T30TXS_2

...

C:\Users\invite\AppData\Local\Temp\ipykernel_5208\35035586.py:38: RuntimeWarning: invalid value encountered in true_divide
  ndvi = (data_b8 - data_b4) / (data_b8 + data_b4)


11  ,  2023-06-30
ndvi_esnandes_2023-06-30.tif
resultat_ndvi
{'driver': 'JP2OpenJPEG', 'dtype': 'uint16', 'nodata': None, 'width': 10980, 'height': 10980, 'count': 1, 'crs': None, 'transform': Affine(10.0, 0.0, 600000.0,
       0.0, -10.0, 5200020.0)}
meta: {'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 10980, 'height': 10980, 'count': 1, 'crs': None, 'transform': Affine(10.0, 0.0, 600000.0,
       0.0, -10.0, 5200020.0)}
out_meta: {'driver': 'Gtiff', 'dtype': 'float32', 'nodata': None, 'width': 196, 'height': 152, 'count': 1, 'crs': None, 'transform': Affine(10.0, 0.0, 642830.0,
       0.0, -10.0, 5125450.0)}


MD5 checksumming:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

File Name                                             Modified             Size
S2B_MSIL2A_20230702T105629_N0509_R094_T30TXS_20230702T122952.SAFE/ 2023-07-02 15:22:56            0
S2B_MSIL2A_20230702T105629_N0509_R094_T30TXS_20230702T122952.SAFE/INSPIRE.xml 2023-07-02 15:22:40        18609
S2B_MSIL2A_20230702T105629_N0509_R094_T30TXS_20230702T122952.SAFE/manifest.safe 2023-07-02 15:22:56        68870
S2B_MSIL2A_20230702T105629_N0509_R094_T30TXS_20230702T122952.SAFE/MTD_MSIL2A.xml 2023-07-02 15:22:40        54866
S2B_MSIL2A_20230702T105629_N0509_R094_T30TXS_20230702T122952.SAFE/GRANULE/ 2023-07-02 15:22:40            0
S2B_MSIL2A_20230702T105629_N0509_R094_T30TXS_20230702T122952.SAFE/HTML/ 2023-07-02 15:22:40            0
S2B_MSIL2A_20230702T105629_N0509_R094_T30TXS_20230702T122952.SAFE/rep_info/ 2023-07-02 15:22:40            0
S2B_MSIL2A_20230702T105629_N0509_R094_T30TXS_20230702T122952.SAFE/DATASTRIP/ 2023-07-02 15:22:40            0
S2B_MSIL2A_20230702T105629_N0509_R094_T30TXS_20230

Done!
12  ,  2023-07-02
ndvi_esnandes_2023-07-02.tif
resultat_ndvi
{'driver': 'JP2OpenJPEG', 'dtype': 'uint16', 'nodata': None, 'width': 10980, 'height': 10980, 'count': 1, 'crs': None, 'transform': Affine(10.0, 0.0, 600000.0,
       0.0, -10.0, 5200020.0)}
meta: {'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 10980, 'height': 10980, 'count': 1, 'crs': None, 'transform': Affine(10.0, 0.0, 600000.0,
       0.0, -10.0, 5200020.0)}
out_meta: {'driver': 'Gtiff', 'dtype': 'float32', 'nodata': None, 'width': 196, 'height': 152, 'count': 1, 'crs': None, 'transform': Affine(10.0, 0.0, 642830.0,
       0.0, -10.0, 5125450.0)}
